# .wav 파일의 총 시간 알려주는 코드

In [ ]:
import numpy as np
import scipy.io as sio
import scipy.io.wavfile
import matplotlib.pyplot as plt

def how_long(Data) :
    sec = []
    for i in range(len(Data)) :
        path = Data[0][i]
        samplerate, data = sio.wavfile.read(path)
        times = np.arange(len(data))/float(samplerate)
        sec.append(times[-1])
    return sum(sec) # 초로 return

In [ ]:
import pandas as pd
data = pd.read_csv("c:/data/deep_voice/label.csv",encoding='CP949',header=None)

for i in range(len(data[[0]])) :
    data[0][i] = 'c:/data/deep_voice/set1/' + data[0][i] # .wav 파일이 있는 폴더

how_long(data)/60 # 분으로 바꿔줌

# csv -> json

In [ ]:
# 0. csv 파일 전부 합치기
data1 = pd.read_csv("c:/data/deep_voice/홍길동.csv",encoding='CP949',header=None)
data2 = pd.read_csv("c:/data/deep_voice/임꺽정.csv",encoding='CP949',header=None)
data3 = pd.read_csv("c:/data/deep_voice/장길산.csv",encoding='CP949',header=None)

result = pd.concat([data2,data3,data1])
result.to_csv("c:/data/deep_voice/result.csv",header=True,index=False)
result

In [ ]:
# 1. csv 파일 전처리
import pandas as pd

result = pd.read_csv("c:/data/deep_voice/음성데이터확장자없는라벨.csv",header=None)
result.dropna(axis=0,inplace=True) # NaN 데이터 행 삭제

result[[0]] = result[[0]].astype(str) # 숫자 데이터 문자열로 변환
result[[0]] = result[[0]].apply(lambda x:x+'.wav',axis=1) # .wav 붙여주기
result.to_csv("c:/data/deep_voice/주희2.csv",header=False,index=False) # 저장

In [ ]:
# csv -> json 만들기 위한 전단계 (dictionary 형태인 json에 맞춰주는 작업)
result
new = pd.DataFrame(data=result.to_numpy().T)
new.columns = new.iloc[0,:]
new.drop(new.index[0]).to_csv("c:/data/deep_voice/new.csv",header=True,index=False)

In [ ]:
# 2. csv -> json
import csv
import json

input_file_name = 'c:/data/deep_voice/new.csv'
output_file_name = 'c:/data/deep_voice/new.txt'

with open(input_file_name, "r", encoding="utf-8", newline="") as input_file, \
        open(output_file_name, "w", encoding="utf-8", newline="") as output_file:
        
    reader = csv.reader(input_file)
    # 첫 줄은 col_names 리스트로 읽어 놓고
    col_names = next(reader)
    # 그 다음 줄부터 zip으로 묶어서 json으로 dumps
    for cols in reader:
        doc = {col_name: col for col_name, col in zip(col_names, cols)}
        print(json.dumps(doc, ensure_ascii=False), file=output_file)

# 무음 기준으로 잘라주는 코드

In [ ]:
'''
* 참고 사이트 *
https://www.thetopsites.net/article/54596077.shtml
https://stackoverflow.com/questions/23730796/using-pydub-to-chop-up-a-long-audio-file
'''

# Import the AudioSegment class for processing audio and the 
# split_on_silence function for separating out silent chunks.
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Define a function to normalize a chunk to a target amplitude.
def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)

# Load your audio.
song = AudioSegment.from_wav("C:/DATA/deep_voice/음성데이터.wav")

# Split track where the silence is 2 seconds or more and get chunks using 
# the imported function.
chunks = split_on_silence (
    # Use the loaded audio.
    song, 
    # Specify that a silent chunk must be at least 2 seconds or 2000 ms long.
    min_silence_len = 400,
    # Consider a chunk silent if it's quieter than -16 dBFS.
    # (You may want to adjust this parameter.)
    silence_thresh = -40
)

# Process each chunk with your parameters
for i, chunk in enumerate(chunks):
    # Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.
    silence_chunk = AudioSegment.silent(duration=100)
    # Add the padding chunk to beginning and end of the entire chunk.
    audio_chunk = silence_chunk + chunk + silence_chunk
    # Normalize the entire chunk.
    normalized_chunk = match_target_amplitude(audio_chunk, -20.0)
    # Export the audio chunk with new bitrate.
    chunk.export("C:/DATA/deep_voice/음성데이터/{0}.wav".format(i), format="wav")